In [1]:
cd ..

/private/home/yannd/projects/NPF


In [2]:
N_THREADS = 8
# Nota Bene : notebooks don't deallocate GPU memory
IS_FORCE_CPU = False # can also be set in the trainer

In [3]:
%autosave 600
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# CENTER PLOTS
from IPython.core.display import HTML
display(HTML(""" <style> .output_png {display: table-cell; text-align: center; margin:auto; }
.prompt display:none;}  </style>"""))

import os
if IS_FORCE_CPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = ""
    
import sys
sys.path.append("notebooks")

import numpy as np
import matplotlib.pyplot as plt
import torch
torch.set_num_threads(N_THREADS)

Autosaving every 600 seconds


In [4]:
from utils.data import get_dataset
mnist_train, mnist_test = get_dataset("mnist")(split="train"), get_dataset("mnist")(split="test")

In [17]:
from neuralproc.architectures.cnn import CNN, ConvBlock
from neuralproc.utils.helpers import channels_to_last_dim
    

In [ ]:
y_train = np.array([y for x, y in iter(mnist_train)])

In [38]:

class Cnn(nn.Module):
    def __init__(self):
        super(Cnn, self).__init__()
        self.fc1 = nn.Linear(1, 100)
        self.conv1 = CNN(100,ConvBlock,Conv=torch.nn.Conv2d,is_chan_last=True)
        self.out = nn.Linear(100, 10)

    def forward(self, x):
        x = channels_to_last_dim(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.conv1(x))
        x = x.view(x.size(0),-1,100).mean(1)
        x = F.softmax(x, dim=-1)
        
        return x

In [39]:
import torch
from torch import nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [40]:
import skorch
from skorch import NeuralNetClassifier

net = NeuralNetClassifier(
    Cnn,
    optimizer=torch.optim.Adam,
    max_epochs=10,
    iterator_train__num_workers=4,
    iterator_valid__num_workers=4,
    lr=0.01,
    device="cuda",
    callbacks=[skorch.callbacks.ProgressBar()],
)

In [41]:
net.fit(mnist_train, y=y_train);

  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        2.2429       0.6141        1.1297  14.6338


      2        0.6834       0.8580        0.4705  14.6030


      3        0.2776       0.9304        0.2485  14.6455


      4        0.2019       0.9098        0.3174  14.7244


      5        0.1743       0.9115        0.3099  14.7454


      6        0.1545       0.9223        0.2683  14.7662


      7        0.1358       0.9244        0.2758  14.7478


      8        0.1230       0.9320        0.2464  14.7745


      9        0.1161       0.9464        0.1862  14.7510


     10        0.1034       0.9594        0.1347  14.7352
